In [1]:
!pip install pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 37.6 MB/s eta 0:00:00


In [2]:
!pip install PyPDF2

import PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.1 MB/s eta 0:00:00


In [4]:
import re
import fitz
def extract_email(email):
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", email)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None

In [5]:
def extract_emails_from_pdf(pdf_file_path):
    doc = fitz.open(pdf_file_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()

    return text

In [3]:
import re

def extract_mobile_number(text):
    phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'), text)

    if phone:
        number = ''.join(phone[0])
        if len(number) > 10:
            return '+' + number
        else:
            return number

In [6]:
!pip install nltk
!python -m nltk nltk.download('words')

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `python -m nltk nltk.download('words')'


In [7]:
pip install spacy

In [8]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
import spacy
from spacy.matcher import Matcher
import fitz  # PyMuPDF

# load pre-trained model
nlp = spacy.load('en_core_web_sm')

# initialize matcher with a vocab
matcher = Matcher(nlp.vocab)

def extract_name_from_pdf(pdf_file_path):
    # Extract text from PDF
    with fitz.open(pdf_file_path) as doc:
        pdf_text = ""
        for page_number in range(doc.page_count):
            page = doc[page_number]
            pdf_text += page.get_text()

    return extract_name(pdf_text)

def extract_name(resume_text):
    nlp_text = nlp(resume_text)

    # First name and Last name are always Proper Nouns
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]

    matcher.add('NAME', patterns=[pattern])

    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

In [17]:
import spacy
import json
from PyPDF2 import PdfReader

class EntityGenerator:
    def __init__(self, text=None):
        self.text = text

    def get(self):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(self.text)
        entities = {"EDUCATION": [], "EXPERIENCE": []}

        # Mots-clés pour identifier les sections
        education_keywords = ["education", "formation", "diplôme", "études"]
        experience_keywords = ["experience", "expérience", "emploi", "travail"]

        current_section = None
        current_phrase = []

        for token in doc:
            if token.text.lower() in education_keywords:
                current_section = "EDUCATION"
            elif token.text.lower() in experience_keywords:
                current_section = "EXPERIENCE"
            elif current_section is not None:
                current_phrase.append(token.text)
                if token.is_punct or token.is_space:
                    sentence = " ".join(current_phrase).strip()
                    # Supprimer les chiffres qui se composent de quatre caractères
                    sentence = ' '.join(word for word in sentence.split() if not word.isdigit() or (len(word) != 4 and not word.startswith("-")))
                    sentence = ''.join(char for char in sentence if char != '-')
                    entities[current_section].append(sentence)
                    current_phrase = []

        return entities

In [18]:
if __name__ == "__main__":
    pdf_file_path = "sample_data/2.pdf"
    pdf_text = extract_emails_from_pdf(pdf_file_path)

    # Assuming the PDF contains email addresses, you can split the text into lines
    lines = pdf_text.split('\n')

    # Extract emails from each line and print the results
    for line in lines:
        extracted_email = extract_email(line)
        extracted_phone = extract_mobile_number(line)
        if extracted_email:
            print(f"Extracted email: {extracted_email}")
        if extracted_phone:
            print(f"Extracted phone: {extracted_phone}")
       # Extract name from PDF
    extracted_name = extract_name_from_pdf(pdf_file_path)

    # Print the extracted name
    print("Extracted Name:", extracted_name)


class Resume:
    def __init__(self, filename=None):
        self.filename = filename

    def get_text_from_pdf(self):
        with open(self.filename, 'rb') as file:
            pdf_reader = PdfReader(file)
            text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
            return text.replace('\n', ' ')  # Remplacer les caractères de nouvelle ligne par un espace

# Example Usage
resume = Resume(filename="/content/sample_data/2.pdf")
resume_text = resume.get_text_from_pdf()

if resume_text:
    entity_generator = EntityGenerator(text=resume_text)
    entities = entity_generator.get()

    # Remove sentences starting with "\u25cb" from the EXPERIENCE section
    entities["EXPERIENCE"] = [sentence for sentence in entities["EXPERIENCE"] if not sentence.startswith("\u25cb")]
    entities["EDUCATION"] = [sentence for sentence in entities["EDUCATION"] if not sentence.startswith("\u25cb")]
    print(json.dumps(entities, indent=3))
else:
    print("Error: No text extracted from the resume.")


Extracted phone: 5059196
Extracted email: benbrahemabir@gmail.com
Extracted Name: Abir Benbrahem
{
   "EDUCATION": [
      "Bachelor in Computer Science ,",
      "",
      "High School Diploma in Experimental Sciences ,"
   ],
   "EXPERIENCE": [
      "Final y ear Pr oject Internship at Medis ,",
      "Initiation Internship at STEG ,"
   ]
}
